In [114]:
# import necessary libraries
import pandas as pd
import re
from airportsdata import load
from actions.done.CreateZFWMessageAction import CreateZFWMessageAction
from actions.done.UpdateEstimatesAction import UpdateEstimatesAction
from actions.done.CalculateWeightAndTrimAction import CalculateWeightAndTrimAction
from actions.done.CreateLoadsheetAction import CreateLoadsheetAction
from actions.done.UpdateFuelDataAction import UpdateFuelDataAction
import json
import os
import concurrent.futures
import time

In [115]:
def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = '../pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "../data"

In [116]:
df = pd.read_parquet(f"{FILE_PATH}/data_parquet/processed_data_combined.parquet", engine=ENGINE)

In [117]:
df.reset_index(inplace=True)

In [118]:
KEEP_COLUMNS = ["flight_id", "action_name","entry_details", "index"]
df = df[KEEP_COLUMNS]

In [119]:
df.rename(columns={"index": "row_id"}, inplace=True)

In [120]:
os.listdir(f"actions/done")
ACTIONS = [action.replace(".py", "") for action in os.listdir(f"actions/done") if action.endswith(".py")]
df = df[df['action_name'].isin(ACTIONS)]

In [121]:
flight_ids = df['flight_id'].unique()
# Set flight_id as index for faster access
df.set_index('flight_id', inplace=True)

In [122]:
def process_flight(flight, df):
    subset_df = df.loc[flight].copy()
    subset_df.sort_values(inplace=True, by="row_id", ascending=True)
    actions = subset_df['action_name'].values
    entry_details = subset_df["entry_details"].values
    temp_estimated_zfws = []
    temp_actual_zfws = []
    temp_actual_fuels = []
    temp_actual_tows = []
    
    for entry, action in zip(entry_details, actions):
        if action == "CreateZFWMessageAction":
            estimated_zfw, actual_zfw = CreateZFWMessageAction(entry)
            if estimated_zfw is not None:
                try:
                    if estimated_zfw != temp_estimated_zfws[-1]:
                        temp_estimated_zfws.append(estimated_zfw)
                except IndexError:
                    temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw is not None:
                try:
                    if actual_zfw != temp_actual_zfws[-1]:
                        temp_actual_zfws.append(actual_zfw)
                except IndexError:
                    temp_actual_zfws.append(actual_zfw)
        elif action == "UpdateEstimatesAction":
            estimated_zfw = UpdateEstimatesAction(entry)
            if estimated_zfw is not None:
                try:
                    if estimated_zfw != temp_estimated_zfws[-1]:
                        temp_estimated_zfws.append(estimated_zfw)
                except IndexError:
                    temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw is not None:
                try:
                    if actual_zfw != temp_actual_zfws[-1]:
                        temp_actual_zfws.append(actual_zfw)
                except IndexError:
                    temp_actual_zfws.append(actual_zfw)
        elif action == "CalculateWeightAndTrimAction":
            actual_zfw = CalculateWeightAndTrimAction(entry)
            if actual_zfw is not None:
                try:
                    if actual_zfw != temp_actual_zfws[-1]:
                        temp_actual_zfws.append(actual_zfw)
                except IndexError:
                    temp_actual_zfws.append(actual_zfw)
        elif action == "CreateLoadsheetAction":
            actual_tow, actual_zfw, estimated_zfw, actual_fuel = CreateLoadsheetAction(entry)
            if actual_zfw is not None:
                try:
                    if actual_zfw != temp_actual_zfws[-1]:
                        temp_actual_zfws.append(actual_zfw)
                except IndexError:
                    temp_actual_zfws.append(actual_zfw)
            if estimated_zfw is not None:
                try:
                    if estimated_zfw != temp_estimated_zfws[-1]:
                        temp_estimated_zfws.append(estimated_zfw)
                except IndexError:
                    temp_estimated_zfws.append(estimated_zfw)
            if actual_fuel is not None:
                try: 
                    if actual_fuel != temp_actual_fuels[-1]:
                        temp_actual_fuels.append(actual_fuel)
                except IndexError:
                    temp_actual_fuels.append(actual_fuel)
            if actual_tow is not None:
                try:
                    if actual_tow != temp_actual_tows[-1]:
                        temp_actual_tows.append(actual_tow)
                except IndexError:
                    temp_actual_tows.append(actual_tow)
        elif action == "UpdateFuelDataAction":
            actual_fuel = UpdateFuelDataAction(entry)
            if actual_fuel is not None:
                try: 
                    if actual_fuel != temp_actual_fuels[-1]:
                        temp_actual_fuels.append(actual_fuel)
                except IndexError:
                    temp_actual_fuels.append(actual_fuel)
    
    return flight, temp_estimated_zfws, temp_actual_zfws, temp_actual_fuels, temp_actual_tows

def collect_results(result):
    flight, temp_estimated_zfws, temp_actual_zfws, temp_actual_fuels, temp_actual_tows = result
    estimated_zfws[flight] = temp_estimated_zfws
    actual_zfws[flight] = temp_actual_zfws
    actual_take_off_fuels[flight] = temp_actual_fuels
    actual_tows[flight] = temp_actual_tows

start = time.time()
estimated_zfws = {}
actual_zfws = {}
actual_take_off_fuels = {}
actual_tows = {}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_flight, flight, df) for flight in flight_ids[:1000]]
    for future in concurrent.futures.as_completed(futures):
        collect_results(future.result())

end = time.time()
print(end - start)


7.025030851364136


In [78]:
len(flight_ids)

14447

In [123]:
estimated_zfws_tuples = [(k, v) for k, v in estimated_zfws.items()]
actual_zfws_tuples = [(k, v) for k, v in actual_zfws.items()]
actual_take_off_fuels_tuples = [(k, v) for k, v in actual_take_off_fuels.items()]
# Create DataFrame
estimated_zfws_df = pd.DataFrame(estimated_zfws_tuples, columns=['flight_number', 'estimated_zfws'])
actual_zfws_df = pd.DataFrame(actual_zfws_tuples, columns=['flight_number', 'actual_zfws'])
actual_take_off_fuels_df = pd.DataFrame(actual_take_off_fuels_tuples, columns=['flight_number', 'actual_take_off_fuels'])
weights = pd.merge(estimated_zfws_df, actual_zfws_df, on='flight_number', how="outer")
weights = pd.merge(weights, actual_take_off_fuels_df, on='flight_number', how="outer")


In [124]:
weights

,flight_number,estimated_zfws,actual_zfws,actual_take_off_fuels
0,MN-1030-2024-1-4,[],"[145679, 127355, 145679]",[]
1,MN-1030-2024-1-5,[145679],"[145679, 148197, 145779, 149631, 146181, 14685...","[48715, 51625]"
2,MN-1030-2024-3-5,"[142264, 145589]","[127355, 142189, 127355, 142264, 142364, 14243...","[47099, 49625]"
3,MN-1030-2024-5-5,"[145051, 149018]","[127355, 145051, 127355, 145051, 145151, 14819...","[4751030, 49125]"
4,MN-1030-2024-8-5,[],[127386],[]
...,...,...,...,...
995,MN-4250-2024-30-4,[],"[49298, 48999, 49116, 48999, 49560, 49298, 489...",[]
996,MN-4250-2024-4-5,[57474],"[56287, 56352, 56287, 56339, 56287, 56339, 562...","[7215, 7758]"
997,MN-4250-2024-5-5,[51158],"[50842, 50549, 50458, 50842, 50666, 50458, 505...","[6964, 7287]"
998,MN-9888-2024-30-4,[],[44617],[]


In [93]:
weights.to_parquet("../data/data_parquet/weights.parquet", engine=ENGINE, compression=COMPRESSION)